# Memory Allocation, Deallocation, and Garbage Collection in Python

Understanding how Python manages memory is crucial for writing efficient applications and avoiding memory leaks. This notebook explains Python's memory management mechanisms.

## What We'll Learn

1. How Python Allocates Memory
2. Reference Counting
3. Garbage Collection
4. The `gc` Module
5. Memory Profiling with `sys.getsizeof()`
6. Memory Optimization Tips
7. Avoiding Memory Leaks

---

## 1. Python Memory Management Basics

Python manages memory automatically using:
1. **Private Heap Space**: All Python objects are stored in a private heap
2. **Memory Manager**: Python's built-in memory manager handles allocation
3. **Reference Counting**: Tracks how many references point to an object
4. **Garbage Collector**: Cleans up circular references

**Key Concepts:**
- You don't manually allocate/deallocate memory (no `malloc`/`free`)
- Python handles it automatically
- Memory is reclaimed when no longer needed
- Objects are never explicitly destroyed

---

## 2. Reference Counting

Python uses **reference counting** as its primary memory management technique. Every object has a reference count that tracks how many references point to it.

**When Reference Count Increases:**
- Object is assigned to a variable
- Object is added to a list, tuple, or dictionary
- Object is passed as an argument

**When Reference Count Decreases:**
- Reference goes out of scope
- Variable is reassigned
- Object is explicitly deleted with `del`
- Container holding the object is destroyed

**When reference count reaches 0, memory is immediately freed.**

In [ ]:
import sys

# Create an object
x = [1, 2, 3]
print(f"Reference count of x: {sys.getrefcount(x)}")  # Note: +1 for getrefcount itself

# Create another reference
y = x
print(f"Reference count after y = x: {sys.getrefcount(x)}")

# Delete a reference
del y
print(f"Reference count after del y: {sys.getrefcount(x)}")

# Add to a list
my_list = [x]
print(f"Reference count after adding to list: {sys.getrefcount(x)}")

# Remove from list
my_list.clear()
print(f"Reference count after clearing list: {sys.getrefcount(x)}")

---

## 3. Garbage Collection

Reference counting alone can't handle **circular references** (objects that reference each other). Python's garbage collector handles this.

**Circular Reference Example:**
```python
a = []
b = []
a.append(b)  # a references b
b.append(a)  # b references a
# Even if we delete a and b, they still reference each other!
```

The **garbage collector** periodically scans for and collects circular references.

In [ ]:
import gc

# Check if garbage collection is enabled
print(f"Garbage collection enabled: {gc.isenabled()}")

# Get garbage collection statistics
print(f"GC stats: {gc.get_stats()}")

# Get count of objects in each generation
print(f"GC counts (gen0, gen1, gen2): {gc.get_count()}")

# Create circular reference
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

# Create circular reference
node1 = Node(1)
node2 = Node(2)
node1.next = node2
node2.next = node1  # Circular!

# Delete references
del node1
del node2

# Force garbage collection
collected = gc.collect()
print(f"\nObjects collected by GC: {collected}")

---

## 4. Measuring Object Memory Size

Use `sys.getsizeof()` to check how much memory an object uses.

In [ ]:
import sys

# Check memory size of different objects
print(f"Integer: {sys.getsizeof(42)} bytes")
print(f"Float: {sys.getsizeof(3.14)} bytes")
print(f"String: {sys.getsizeof('Hello')} bytes")
print(f"Empty list: {sys.getsizeof([])} bytes")
print(f"List with 10 items: {sys.getsizeof([1,2,3,4,5,6,7,8,9,10])} bytes")
print(f"Empty dict: {sys.getsizeof({})} bytes")
print(f"Dict with 5 items: {sys.getsizeof({i: i for i in range(5)})} bytes")

# Comparison: List vs Tuple
my_list = [1, 2, 3, 4, 5]
my_tuple = (1, 2, 3, 4, 5)
print(f"\nList: {sys.getsizeof(my_list)} bytes")
print(f"Tuple: {sys.getsizeof(my_tuple)} bytes (smaller!)")

---

## 5. Memory Optimization Tips

**1. Use Generators Instead of Lists** (for large datasets)

In [ ]:
import sys

# List - stores all values in memory
large_list = [x for x in range(1000000)]
print(f"List memory: {sys.getsizeof(large_list):,} bytes")

# Generator - generates values on demand
large_gen = (x for x in range(1000000))
print(f"Generator memory: {sys.getsizeof(large_gen):,} bytes")

print(f"\nMemory saved: {sys.getsizeof(large_list) - sys.getsizeof(large_gen):,} bytes!")

---

## Summary

**Key Takeaways:**

1. **Automatic Memory Management**: Python handles memory automatically
2. **Reference Counting**: Primary mechanism - objects freed when count reaches 0
3. **Garbage Collection**: Handles circular references that reference counting can't
4. **gc Module**: Control and monitor garbage collection
5. **sys.getsizeof()**: Measure object memory usage

**Memory Management Mechanisms:**

| Mechanism | Handles | Limitation |
|-----------|---------|------------|
| Reference Counting | Most objects | Can't handle cycles |
| Garbage Collector | Circular references | Periodic, has overhead |

**Optimization Tips:**
1. Use generators for large sequences
2. Use `__slots__` in classes to reduce memory
3. Use tuples instead of lists when data is immutable
4. Delete large objects when done (`del`)
5. Use `gc.collect()` to force garbage collection if needed
6. Avoid circular references when possible

**Common Memory Issues:**
- Memory leaks from circular references
- Keeping references to large objects unintentionally
- Creating too many small objects (use object pooling)
- Not closing file handles/database connections

Understanding memory management helps write efficient, scalable Python applications!